## Install

!pip install  fuzzywuzzy

## Import

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm
import re

In [5]:
# Load the first CSV file into a Pandas dataframe
df1 = pd.read_csv("./data/output/sample_data_Scrape_Result.csv")
# Load the second CSV file into a second Pandas dataframe
df2 = pd.read_csv("./data/input/Globe-institution_data.csv")

In [6]:
df1.head(5)


,Unnamed: 0,Acc,ID,Center Names,Submitted_by,Biosample_Submission
0,0,SRR8606903,7314038,AUSTRALIAN INSTITUTE OF MARINCE SCIENCES,Australian Institute of Marince sciences,"Australian Institute of Marince sciences, Patr..."
1,1,SRR11788653,10848896,CLARK UNIVERSITY,Clark University,"Clark University, Nathan Ahlgren"
2,2,SRR9158324,7959983,CALIFORNIA STATE UNIVERSITY FULLERTON,California State University Fullerton,"California State University Fullerton, Maria R..."
3,3,SRR11881309,10964535,"CALIFORNIA STATE UNIVERSITY, FULLERTON","California State University, Fullerton","California State University, Fullerton, Joshua..."
4,4,SRR10293930,9209919,"CALIFORNIA STATE UNIVERSITY, LOS ANGELES","California State University, Los Angeles","California State University, Los Angeles, Kirs..."


In [12]:
# List to hold the results
results = []
# Set to hold the 'Acc' values for which matches have already been found
matched_ids = set()
# Iterate over the columns in df1
for column in tqdm(['Center Names', 'Submitted_by', 'Biosample_Submission'], desc='Columns'):
    # Iterate over the rows in df1
    for index, row in tqdm(df1.iterrows(), total=df1.shape[0], desc='Rows', leave=False):
        # Check if this 'Acc' has already been matched
        if row['Acc'] in matched_ids:
            continue
        # Convert the series to a string
        string = str(row[column])
        # Check if the column contains a comma
        if ',' in string:
            # Split the string into two parts based on the comma
            parts = string.split(',')
            # Find the closest match in df2 for each part
            matches = [process.extractOne(part.strip(), df2['Institution'], scorer=fuzz.token_sort_ratio) for part in parts]
            # Check if there is a match for both parts
            if all(match and match[1] >= 70 for match in matches):
                # Get the indices of the matching rows in df2
                match_indices = [df2[df2['Institution'] == match[0]].index[0] for match in matches]
                # Concatenate the rows from df1 and df2 and add to the results list
                result = pd.concat([df1.loc[[index], :].reset_index(drop=True), df2.loc[match_indices, :].reset_index(drop=True)], axis=1)
                result['Score'] = sum(match[1] for match in matches) / len(matches)
                results.append(result)
                # Add the 'Acc' value to the set of matched ids
                matched_ids.add(row['Acc'])
        else:
            # Find the closest match in df2
            match = process.extractOne(string, df2['Institution'], scorer=fuzz.token_sort_ratio)
            if match and match[1] >= 70:
                # Get the index of the matching row in df2
                match_index = df2[df2['Institution'] == match[0]].index[0]
                # Concatenate the rows from df1 and df2 and add to the results list
                result = pd.concat([df1.loc[[index], :].reset_index(drop=True), df2.loc[[match_index], :].reset_index(drop=True)], axis=1)
                result['Score'] = match[1]
                results.append(result)
                # Add the 'Acc' value to the set of matched ids
                matched_ids.add(row['Acc'])
            else:
                # If no match is found or the Score is below 70%, add the row from df1 to the results list
                result = df1.loc[[index], :].reset_index(drop=True)
                result['Score'] = None
                results.append(result)
# Concatenate all the results into a single dataframe
final_result = pd.concat(results)

Columns: 100%|██████████| 3/3 [00:02<00:00,  1.01it/s]


In [23]:
final_result.head(5)


,Unnamed: 0,Acc,ID,Center Names,Submitted_by,Biosample_Submission,Institution,Country/territory,Score
0,0.0,SRR8606903,7314038.0,AUSTRALIAN INSTITUTE OF MARINCE SCIENCES,Australian Institute of Marince sciences,"Australian Institute of Marince sciences, Patr...",Australian Institute of Marine Science (AIMS),Australia,92.0
0,1.0,SRR11788653,10848896.0,CLARK UNIVERSITY,Clark University,"Clark University, Nathan Ahlgren",Clark University,United States of America (USA),100.0
0,2.0,SRR9158324,7959983.0,CALIFORNIA STATE UNIVERSITY FULLERTON,California State University Fullerton,"California State University Fullerton, Maria R...","California State University, Fullerton (CSUF)",United States of America (USA),94.0
0,6.0,SRR10018586,8900739.0,CFSAN,FDA Center for Food Safety and Applied Nutrit...,CFSAN,NaN,NaN,NaN
0,8.0,SRR10406092,9345478.0,AUSTRALIAN INSTITUTE OF MARINE SCIENCE,Australian Institute of Marine Science,"Australian Institute of Marine Science, Heidi ...",Australian Institute of Marine Science (AIMS),Australia,94.0


# Split the results into separate dataframes based on the Score score

In [14]:
above_90 = final_result[final_result['Score'] >= 90]
above_80 = final_result[(final_result['Score'] >= 80) & (final_result['Score'] < 90)]
above_70 = final_result[(final_result['Score'] >= 70) & (final_result['Score'] < 80)]
below_70 = final_result[final_result['Score'] < 70]
below_69 = below_70[below_70['Score'] < 69]

# Check if the number of unique 'Acc' values in final_result matches the number of rows in df1

In [15]:
if final_result['Acc'].nunique() == df1.shape[0]:
    print("Result contains a row for every 'Acc' in df1")
else:
    print("Result does not contain a row for every 'Acc' in df1")

Result does not contain a row for every 'Acc' in df1


In [17]:
above_90.count()


Unnamed: 0              12
Acc                     12
ID                      12
Center Names            12
Submitted_by            12
Biosample_Submission    12
Institution             12
Country/territory       12
Score                   12
dtype: int64

In [18]:
above_80.count()

Unnamed: 0              6
Acc                     6
ID                      6
Center Names            6
Submitted_by            6
Biosample_Submission    6
Institution             6
Country/territory       6
Score                   6
dtype: int64

In [19]:
above_70.count()

Unnamed: 0              10
Acc                     10
ID                      10
Center Names            10
Submitted_by            10
Biosample_Submission    10
Institution             11
Country/territory       11
Score                   11
dtype: int64

In [21]:
below_69.count()

Unnamed: 0              0
Acc                     0
ID                      0
Center Names            0
Submitted_by            0
Biosample_Submission    0
Institution             0
Country/territory       0
Score                   0
dtype: int64

# Save each dataframe to a separate CSV file

In [24]:
above_90.to_csv('./data/output/sample_data_above_90.csv', index=False, encoding='utf-8')
above_80.to_csv('./data/output/sample_data_above_80.csv', index=False, encoding='utf-8')
above_70.to_csv('./data/output/sample_data_above_70.csv', index=False, encoding='utf-8')
below_70.to_csv('./data/output/sample_data_below_69.csv', index=False, encoding='utf-8')

# Save the final result

In [25]:
final_result.to_csv('./data/output/sample_data_Fuzzy_result.csv', encoding='utf-8')